In [155]:
import pandas as pd
from math import sqrt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [156]:
artistlist_df = pd.read_csv("lastfm_artist_list.csv")
scrobblelist_df = pd.read_csv("lastfm_user_scrobbles.csv")
print(artistlist_df.info)
scrobblelist_df.info()
scrobblelist_df["artist_id"].value_counts()


<bound method DataFrame.info of        artist_id   artist_name
0              1       __Max__
1              2       _Algol_
2              3     -123 Min.
3              4          -Oz-
4              5  -T De Sangre
...          ...           ...
17488      17489          鷺巣詩郎
17489      17490           黃立行
17490      17491           黄义达
17491      17492         黒木メイサ
17492      17493            鼓童

[17493 rows x 2 columns]>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92792 entries, 0 to 92791
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    92792 non-null  int64
 1   artist_id  92792 non-null  int64
 2   scrobbles  92792 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


8460     611
2381     522
12276    485
14340    480
8014     473
        ... 
17343      1
5824       1
1390       1
4866       1
8660       1
Name: artist_id, Length: 17493, dtype: int64

In [157]:
scaler = MinMaxScaler(feature_range= (1,10))
scaler.fit(scrobblelist_df[["scrobbles","scrobbles"]])

newdata = scaler.transform(scrobblelist_df[["scrobbles","scrobbles"]])
scrobblelist_df['ratings']=newdata[:,0]
scrobblelist_df.drop("scrobbles",axis= 1,inplace= True)
print(scrobblelist_df)

       user_id  artist_id   ratings
0            1       4562  1.354236
1            1      10191  1.298276
2            1        494  1.289625
3            1       6673  1.262806
4            1       8402  1.229200
...        ...        ...       ...
92787     1892      10098  1.007068
92788     1892       8660  1.006686
92789     1892       3274  1.006558
92790     1892       4240  1.005895
92791     1892      10842  1.005793

[92792 rows x 3 columns]


In [158]:
userInput = [{"artist_name": "Lady Gaga" ,"ratings": 5},
{"artist_name": "Britney Spears","ratings": 8},
{"artist_name": "Rihanna","ratings": 10},
{"artist_name": "The Beatles","ratings": 6},
{"artist_name": "Katy Perry","ratings": 9}]

inputArtists = pd.DataFrame(userInput)
print(inputArtists)

      artist_name  ratings
0       Lady Gaga        5
1  Britney Spears        8
2         Rihanna       10
3     The Beatles        6
4      Katy Perry        9


In [159]:
inputId= artistlist_df[artistlist_df["artist_name"].isin(inputArtists["artist_name"].tolist())]
inputArtists = pd.merge(inputId, inputArtists)
inputArtists = inputArtists[["artist_id","artist_name","ratings"]]
print(inputArtists)
#print(artistlist_df[artistlist_df['artist_name']=="Lady Gaga"])
#print(artistlist_df["artist_name"])
# inputArtists = pd.merge(inputId, inputArtists)
# print(inputArtists)


#inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#inputMovies = pd.merge(inputId, inputMovies)
#inputMovies = inputMovies.drop('year', 1) #we don't really need this at the moment
#inputMovies = inputMovies[['movieId','title','rating']]
#print(inputMovies)

   artist_id     artist_name  ratings
0       2381  Britney Spears        8
1       8014      Katy Perry        9
2       8460       Lady Gaga        5
3      12276         Rihanna       10
4      14340     The Beatles        6


In [160]:
userSubset = scrobblelist_df[scrobblelist_df["artist_id"].isin(inputArtists["artist_id"].tolist())]
print(userSubset.groupby('artist_id').count())

           user_id  ratings
artist_id                  
2381           522      522
8014           473      473
8460           611      611
12276          485      485
14340          480      480


In [161]:
userSubsetgroup = userSubset.groupby(["user_id"])
def take_5_elem(x):
    return len(x[1])

userSubsetgroup = sorted(userSubsetgroup, key=take_5_elem, reverse=True)
userSubsetgroup = userSubsetgroup[0:100]
print(userSubsetgroup[0:5])

[(46,       user_id  artist_id   ratings
2224       46       8460  1.002832
2234       46       8014  1.000944
2239       46       2381  1.000561
2248       46      14340  1.000357
2252       46      12276  1.000306), (94,       user_id  artist_id   ratings
4613       94       2381  1.063769
4616       94      14340  1.007221
4618       94       8014  1.006941
4620       94       8460  1.005665
4622       94      12276  1.004491), (170,       user_id  artist_id   ratings
8363      170       8460  1.108756
8364      170       2381  1.098294
8390      170      14340  1.004772
8397      170      12276  1.004108
8406      170       8014  1.002143), (203,       user_id  artist_id   ratings
9947      203       2381  1.022940
9959      203      12276  1.007119
9964      203      14340  1.005997
9965      203       8460  1.004797
9975      203       8014  1.003164), (226,        user_id  artist_id   ratings
11094      226       8460  1.033581
11099      226       2381  1.010692
11101      226 

In [162]:
pearCorrDict = {}
for name, group in userSubsetgroup:
    group = group.sort_values(by="artist_id")
    inputArtists = inputArtists.sort_values(by="artist_id")
    
    nRatings = len(group)
    
    temp_df = inputArtists[inputArtists["artist_id"].isin(group["artist_id"].tolist())]
    
    tempRatingList = temp_df['ratings'].tolist()
    tempGroupList = group["ratings"].tolist()
    
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearCorrDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearCorrDict[name] = 0
    

In [163]:
pearDF = pd.DataFrame.from_dict(pearCorrDict, orient= "index")
pearDF.columns = ["similarityIndex"]
pearDF["user_id"] = pearDF.index
pearDF.index = range(len(pearDF))
print(pearDF.head())

   similarityIndex  user_id
0        -0.646502       46
1         0.092070       94
2        -0.523865      170
3         0.129370      203
4        -0.587149      226


In [164]:
topUsers= pearDF.sort_values(by="similarityIndex", ascending = False)[0:50]
print(topUsers.head())

    similarityIndex  user_id
53         0.694234      141
87         0.670170      366
50         0.639384      129
36         0.631580       44
88         0.629080      373


In [169]:
topUsersRating = topUsers.merge(scrobblelist_df, left_on="user_id", right_on="user_id", how="inner")
print(topUsersRating.head(100))

    similarityIndex  user_id  artist_id   ratings
0          0.694234      141      12276  1.608060
1          0.694234      141       8014  1.138025
2          0.694234      141       2134  1.114957
3          0.694234      141      17234  1.114294
4          0.694234      141       1820  1.109471
..              ...      ...        ...       ...
95         0.670170      366       2056  1.001199
96         0.670170      366       9226  1.001199
97         0.670170      366       9725  1.001199
98         0.670170      366      14259  1.001199
99         0.670170      366      11950  1.001174

[100 rows x 4 columns]


In [172]:
topUsersRating["weightedRating"] = topUsersRating["similarityIndex"]*topUsersRating["ratings"]
print(topUsersRating.head())

   similarityIndex  user_id  artist_id   ratings  weightedRating
0         0.694234      141      12276  1.608060        1.116370
1         0.694234      141       8014  1.138025        0.790055
2         0.694234      141       2134  1.114957        0.774041
3         0.694234      141      17234  1.114294        0.773580
4         0.694234      141       1820  1.109471        0.770232


In [173]:
tempTopUsersRating = topUsersRating.groupby('artist_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

           sum_similarityIndex  sum_weightedRating
artist_id                                         
28                    0.698249            0.699212
48                   -0.002274           -0.002274
52                    0.383974            0.384621
96                    0.223452            0.246351
99                    0.223452            0.226816


In [175]:
recommend_df = pd.DataFrame()
recommend_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommend_df['artist_id'] = tempTopUsersRating.index
print(recommend_df.head(100))

           weighted average recommendation score  artist_id
artist_id                                                  
28                                      1.001379         28
48                                      1.000051         48
52                                      1.001684         52
96                                      1.102479         96
99                                      1.015055         99
...                                          ...        ...
2294                                    1.008906       2294
2311                                    1.015247       2311
2329                                    1.009416       2329
2370                                    1.001250       2370
2381                                    1.031093       2381

[100 rows x 2 columns]


In [176]:
recommend_df = recommend_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommend_df)


           weighted average recommendation score  artist_id
artist_id                                                  
12276                                   1.236922      12276
857                                     1.229404        857
6644                                    1.187886       6644
10750                                   1.169208      10750
11894                                   1.121454      11894
...                                          ...        ...
2145                                    0.897217       2145
8636                                    0.860463       8636
9696                                    0.857575       9696
16319                                   0.826566      16319
8440                                    0.588036       8440

[749 rows x 2 columns]


In [178]:
recommend_artist=artistlist_df.loc[artistlist_df['artist_id'].isin(recommend_df['artist_id'])]

#we don't want to recommend the same movie
recommend_artist=recommend_artist.loc[~recommend_artist.artist_id.isin(userSubset['artist_id'])]

print(recommend_artist)

       artist_id artist_name
27            28      *Nsync
47            48         110
51            52   12 Stones
95            96        2ne1
98            99         2pm
...          ...         ...
17365      17366       安室奈美恵
17398      17399   放課後ティータイム
17409      17410        東方神起
17433      17434       浜崎あゆみ
17491      17492       黒木メイサ

[744 rows x 2 columns]
